In [5]:
import numpy as np
import os
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, Flatten, BatchNormalization, MaxPooling2D, Dropout, Input
from keras import metrics
from keras.models import model_from_json
from keras.callbacks import Callback
from keras import optimizers
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, accuracy_score

In [10]:
input_folder = "./input"
op_folder = "./results"

In [7]:
# X_train = np.load(input_folder + "/X_train.npy")
# y_train = np.load(input_folder + "/y_train.npy")
# X_test = np.load(input_folder + "/X_test.npy")
# y_test = np.load(input_folder + "/y_test.npy")
# print (X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [9]:
inp = Input(shape=(720,1280,3))
cnn = Conv2D(64, kernel_size=10, strides=5, activation="relu")(inp)
cnn = BatchNormalization()(cnn)
cnn = MaxPooling2D(pool_size=4, strides=2)(cnn)
cnn = Conv2D(8, kernel_size=1, strides=1, activation="relu")(cnn)
cnn = BatchNormalization()(cnn)
cnn = Dropout(rate=0.4)(cnn)
x = Flatten()(cnn)

x = Dense(512, activation="relu")(x)
x = Dropout(0.4)(x)
x = BatchNormalization()(x)
x = Dense(1, activation="sigmoid", name="scene")(x)

model = Model(inputs=inp, outputs=[x])

model.compile(optimizer="adam",
              loss={"scene": "binary_crossentropy"},
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 720, 1280, 3)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 143, 255, 64)      19264     
_________________________________________________________________
batch_normalization_4 (Batch (None, 143, 255, 64)      256       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 70, 126, 64)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 70, 126, 8)        520       
_________________________________________________________________
batch_normalization_5 (Batch (None, 70, 126, 8)        32        
_________________________________________________________________
dropout_3 (Dropout)          (None, 70, 126, 8)        0         
__________

In [ ]:
history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=2, batch_size=100)
print ("after train")
if not os.path.exists(op_folder):
    os.makedirs(op_folder)
model.save_weights(op_folder + "/model_hackathon.h5")
# print ("Model saved...")

# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='lower right')
plt.savefig("train_acc.png", bbox_inches='tight')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper right')
plt.savefig("train_loss.png", bbox_inches='tight')
plt.show()
